# KEN1435 - Principles of Data Science | Homework Assignment 3 (HA3) :  Societal Mood

For this assignment, we will reconsider the data set that is extracted from social media. As previously introduced, it covers a collection of users on social media that were retrieved based on posts that referenced a list of keywords related to the COVID-19 pandemic. Among all messages acquired with these keywords, those users were extacted that indicated their location was a city in the Netherlands. Specifically, we take a look at the accounts that specified "Maastricht" as their location.

First we load the necessary python packages

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import norm

tab10 = plt.get_cmap("tab10").colors

### Exercise 1. (*1pt*)
Load the data set as provided in the files `users_HA3.tsv` and `tweets_HA3.tsv` into the variables `users` and `tweets`, respectively. Make sure that you convert the times at which the tweets are created from strings into a usable format.

#### Solution

In [8]:
users = pd.read_csv("users_HA3.tsv", sep="\t")
tweets = pd.read_csv("tweets_HA3.tsv", sep= "\t")
users

,Unnamed: 0,gender,gender.1,twitter,twitter.1
0,NaN,male,female,followers,friends
1,user_id,NaN,NaN,NaN,NaN
2,043u0001,0.9428,0.0572,733,894
3,043u0003,0.9649,0.0351,92,647
4,043u0006,0.7454,0.2546,276,278
5,043u0015,0.0009,0.9991,8576,5726
6,043u0016,0.0073,0.9927,42,382
7,043u0017,0.9971,0.0029,699,1130
8,043u0018,1.0,0.0,737,666
9,043u0019,0.9953,0.0047,1344,1165


## Differences between male and female accounts
When processing the accounts that were collected in the data gathering phase, the researchers also analyzed the profile pages of the accounts using a gender classifier. The file `users_HA3.tsv` contains the outcomes of this classification. For each account, the classifier outputs a probability that an account would be classified with either label.

The researchers want to analyze the difference in expressed mood between male and female accounts. They decide that they will use a cut-off at a probability of `0.8` as an inclusion criterium. So if an account scores more than `0.8` on either male or female probability, it is included in that group for further analysis.

### Exercise 2. (*3pt*)
Visualize the ANEW Valence distribution of all tweets from the accounts that are classified as male and female.

#### Solution

Based on the visual comparison above, the researchers want to determine whether the male and female averages align. Therefore, they run a one sided statistical test.

### Exercise 3. (*2pt*)
Denote the null-hypothesis and alternative hypothesis for this test.

#### Solution

- $H_0$ (null-hypothesis): ***YOUR ANSWER GOES HERE***.
- $H_a$ (alternative hypothesis): ***YOUR ANSWER GOES HERE***.

Suppose that we may assume that the average ANEW Valence score for female accounts follows a normal distribution. We determine the mean and standard deviation of the distribution directly from the data.

### Exercise 4. (*2pt*)
Determine the value of the test-statictic and the $P$-value.

#### Solution

## Response to COVID-19
To assess each accounts reaction to the start of the COVID-19 pandemic, we will investigate the difference the average ANEW Valence in the month before the first COVID-19 case was observed in the Netherlands (i.e., December 2019) and a month after (i.e., March 2020).

### Exercise 5. (*2pt*)
Which accounts have the biggest and lowest difference in average ANEW Valence across these two months?

#### Solution